In [1]:
from numpy import *
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float, curLine)
        dataMat.append(fltLine)
    return dataMat

In [2]:
# 欧式距离
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2)))
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k, n)))
    for j in range(n): 
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k,1)
    return centroids

In [6]:
# K-均值聚类算法
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet,k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = -1
#             寻找最近的质心
            for j in range(k):
                distJI = distMeas(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print centroids
#         更新质心位置
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

In [7]:
datMat = mat(loadDataSet('testSet.txt'))
kMeans(datMat,4)

[[ 2.85206201  2.09623935]
 [-1.7078761  -0.46465781]
 [ 3.73818476 -3.99911099]
 [-3.50876556  0.11578367]]
[[ 2.46383071  3.1198451 ]
 [-2.11104069 -2.05819462]
 [ 2.91339042 -2.85004268]
 [-3.25705152  0.73091444]]
[[ 2.46383071  3.1198451 ]
 [-3.1877719  -2.82981938]
 [ 2.91339042 -2.85004268]
 [-2.54951105  2.75812458]]
[[ 2.6265299   3.10868015]
 [-3.38237045 -2.9473363 ]
 [ 2.80293085 -2.7315146 ]
 [-2.46154315  2.78737555]]


(matrix([[ 2.6265299 ,  3.10868015],
         [-3.38237045, -2.9473363 ],
         [ 2.80293085, -2.7315146 ],
         [-2.46154315,  2.78737555]]),
 matrix([[  0.00000000e+00,   2.32019150e+00],
         [  3.00000000e+00,   1.39004893e+00],
         [  2.00000000e+00,   6.63839104e+00],
         [  1.00000000e+00,   4.16140951e+00],
         [  0.00000000e+00,   2.76967820e+00],
         [  3.00000000e+00,   2.80101213e+00],
         [  2.00000000e+00,   5.85909807e+00],
         [  1.00000000e+00,   1.50646425e+00],
         [  0.00000000e+00,   2.29348924e+00],
         [  3.00000000e+00,   6.45967483e-01],
         [  2.00000000e+00,   1.74010499e+00],
         [  1.00000000e+00,   3.77769471e-01],
         [  0.00000000e+00,   2.51695402e+00],
         [  3.00000000e+00,   1.38716420e-01],
         [  2.00000000e+00,   9.47633071e+00],
         [  1.00000000e+00,   9.97310599e+00],
         [  0.00000000e+00,   2.39726914e+00],
         [  3.00000000e+00,   3.10242360e+00],
    

In [14]:
# 二分K-均值聚类
def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
#     创建一个初始簇
    centroid0 = mean(dataSet, axis=0).tolist()[0]  
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:,1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A != i)[0],1])
            print "sseSplit, and notSplit: ",sseSplit, sseNotSplit
            if(sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print 'the bestCentToSplit is: ', bestCentToSplit
        print 'the len of bestClustAss is: ', len(bestClustAss)
        centList[bestCentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return centList, clusterAssment

In [15]:
dataMat3 = mat(loadDataSet('testSet2.txt'))
centList, myNewAssments = biKmeans(dataMat3, 3)

[[-3.95373386 -3.48356518]
 [ 4.02186129  1.54536113]]
[[-1.69031148 -0.38855548]
 [ 1.71544126  3.19785696]]
[[-1.71263297 -0.19674017]
 [ 2.01915156  3.21622852]]
[[-1.72153338 -0.00938424]
 [ 2.35797261  3.21160974]]
[[-1.73028592  0.20133246]
 [ 2.76275171  3.12704005]]
[[-1.70351595  0.27408125]
 [ 2.93386365  3.12782785]]
sseSplit, and notSplit:  541.297629265 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  60
[[-0.71942501 -3.72310313]
 [-1.43685595 -1.55910388]]
[[-0.11329936 -3.20075614]
 [-2.55978642  2.14514754]]
[[-0.45965615 -2.7782156 ]
 [-2.94737575  3.3263781 ]]
sseSplit, and notSplit:  67.2202000798 39.5292986821
[[ 3.96524991  3.14097997]
 [ 3.48002996  4.25639244]]
[[ 3.47582455  2.59709691]
 [ 2.271467    3.776499  ]]
sseSplit, and notSplit:  25.4640400549 501.768330583
the bestCentToSplit is:  0
the len of bestClustAss is:  40


In [16]:
centList

[matrix([[-0.45965615, -2.7782156 ]]),
 matrix([[ 2.93386365,  3.12782785]]),
 matrix([[-2.94737575,  3.3263781 ]])]